# Laboratorio de regresión - 4

|                |   |
:----------------|---|
| **Nombre**    Camila Garcia |   |
| **Fecha**    10 de septiembre  |   |
| **Expediente 749725** |   |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente. 

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

Compara los resultados de los parámetros y sus *p-values*, y los $R^2$ resultantes.

In [3]:
import pandas as pd
import statsmodels.api as sm

data = pd.read_csv("Advertising.csv")

X = data[["TV", "radio", "newspaper"]]
y = data["sales"].astype(float)

Xc = sm.add_constant(X)
ols = sm.OLS(y, Xc).fit()

print(ols.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     570.3
Date:                Thu, 11 Sep 2025   Prob (F-statistic):           1.58e-96
Time:                        12:28:08   Log-Likelihood:                -386.18
No. Observations:                 200   AIC:                             780.4
Df Residuals:                     196   BIC:                             793.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.9389      0.312      9.422      0.0

In [5]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

data = pd.read_csv("Advertising.csv")
X = data[["TV", "radio", "newspaper"]]
y = data["sales"].astype(float)

alpha_ridge = 1.0  
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", Ridge(alpha=alpha_ridge, random_state=0))
]).fit(X, y)

r2 = r2_score(y, pipe.predict(X))

scaler = pipe.named_steps["scaler"]
ridge  = pipe.named_steps["ridge"]

betas_std = ridge.coef_
sigma = scaler.scale_
muX = scaler.mean_
muy = y.mean()

betas = betas_std / sigma
intercepto = float(muy - np.dot(betas, muX))

print(f"Ridge (L2), alpha={alpha_ridge}")
print(f"R^2: {r2:.4f}")
print("Parámetros (escala original):")
print(f"  Intercepto: {intercepto:.6f}")
for name, b in zip(["TV","radio","newspaper"], betas):
    print(f"  beta_{name}: {b:.6f}")

Ridge (L2), alpha=1.0
R^2: 0.8972
Parámetros (escala original):
  Intercepto: 2.986206
  beta_TV: 0.045542
  beta_radio: 0.187507
  beta_newspaper: -0.000737


In [7]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score

data = pd.read_csv("Advertising.csv")
X = data[["TV", "radio", "newspaper"]]
y = data["sales"].astype(float)

alpha_lasso = 0.1 
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("lasso", Lasso(alpha=alpha_lasso, max_iter=10000, random_state=0))
]).fit(X, y)

r2 = r2_score(y, pipe.predict(X))

scaler = pipe.named_steps["scaler"]
lasso  = pipe.named_steps["lasso"]

betas_std = lasso.coef_
sigma = scaler.scale_
muX = scaler.mean_
muy = y.mean()

betas = betas_std / sigma
intercepto = float(muy - np.dot(betas, muX))

print(f"Lasso (L1), alpha={alpha_lasso}")
print(f"R^2: {r2:.4f}")
print("Parámetros (escala original):")
print(f"  Intercepto: {intercepto:.6f}")
for name, b in zip(["TV","radio","newspaper"], betas):
    print(f"  beta_{name}: {b:.6f}")


Lasso (L1), alpha=0.1
R^2: 0.8965
Parámetros (escala original):
  Intercepto: 3.232802
  beta_TV: 0.044648
  beta_radio: 0.181593
  beta_newspaper: 0.000000
